In [1]:
print("hello")

hello


In [2]:
from langchain_groq import ChatGroq

e:\Dec 2025\chatbot\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
import os

In [44]:
api_key=os.getenv('groq_api_key')
os.environ["groq_api_key"]=api_key
pinecone_key=os.getenv("PINECONE_API_KEY")
os.environ['PINECONE_API_KEY']=pinecone_key


In [3]:

model="llama-3.1-8b-instant"
groq_model=ChatGroq(model=model)

In [4]:
kk=groq_model.invoke("what is the capital of france?")
print(kk)

content='The capital of France is Paris.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 42, 'total_tokens': 50, 'completion_time': 0.007352272, 'completion_tokens_details': None, 'prompt_time': 0.002026942, 'prompt_tokens_details': None, 'queue_time': 0.055701168, 'total_time': 0.009379214}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--019b4c9f-740c-7173-b8d8-17635e3eba2c-0' usage_metadata={'input_tokens': 42, 'output_tokens': 8, 'total_tokens': 50}


In [5]:
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
import os

In [6]:
def load_pdf_file(data):
    loader=DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    document=loader.load()
    return document

In [7]:
extracted_docs=load_pdf_file('data')

In [8]:
from typing import List
from langchain_core.documents import Document

In [9]:
def filter_to_minimal(docs:  List[Document]) -> List[Document]:
    minimal_docs :List[Document]= []
    for doc in docs:
        src=doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source":src}
            )
        )
    return minimal_docs


In [10]:
minimal_docs=filter_to_minimal(extracted_docs)

In [11]:
text_split=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=500)

In [12]:
splits=text_split.split_documents(minimal_docs)

In [15]:
pinecone_key=os.getenv("PINECONE_API_KEY")

In [16]:
from pinecone import Pinecone,ServerlessSpec
pine_cone_api=pinecone_key

pc=Pinecone(api_key=pinecone_key)

In [14]:
index_name="quickstart2"


In [17]:
from pinecone import ServerlessSpec

index_name="medicalchatbot22"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws",region="us-east-1")

    )



In [18]:
index=pc.Index(index_name)

In [19]:
index

In [23]:
from langchain_pinecone import PineconeVectorStore

In [25]:
embedding_model=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")

In [26]:
docsearch=PineconeVectorStore.from_documents(
    documents=minimal_docs,
    embedding=embedding_model,
    index_name=index_name
)

In [27]:
retriver=docsearch.as_retriever(search_type='similarity',search_kwargs={'k':3})


In [28]:
retriver_doc=retriver.invoke("what is sex?")

In [29]:
from langchain_core.prompts import ChatPromptTemplate

In [35]:
RAG_PROMPT_CITATION = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a medical ai -bot.
Answer strictly using the provided context.

Rules:
- Cite facts using [Source X].
- If unsure, say you do not know.
- Do not fabricate information.
"""
        ),
        (
            "human",
            """Context:
{context}

Question:
{question}

Answer (with citations):"""
        ),
    ]
)


In [36]:
def format_docs(docs):
    return "\n\n".join(
            d.page_content for d in docs)
    


In [37]:
from langchain_core.runnables import RunnablePassthrough

In [38]:
rag_chain = (
    {
        "context": retriver|format_docs,
        "question": RunnablePassthrough()
    }
    | RAG_PROMPT_CITATION
    | groq_model
)


In [40]:
rag_chain.invoke("what is health").content

'Health is a complex and multifaceted concept that encompasses physical, mental, emotional, and social well-being. According to the World Health Organization (WHO), health is defined as "a state of complete physical, mental, and social well-being and not merely the absence of disease or infirmity" [Source 1].\n\nThe WHO further breaks down health into several key components, including:\n\n1. Physical health: refers to the body\'s overall physical functioning, including the absence of disease or injury [Source 2].\n2. Mental health: encompasses a person\'s emotional and psychological well-being, including their ability to cope with stress, anxiety, and other mental health challenges [Source 3].\n3. Social health: involves a person\'s relationships with others, their sense of belonging, and their ability to participate in social activities [Source 4].\n\nIn addition to these components, health also includes the ability to adapt to changes and challenges, as well as the ability to maintai

In [45]:
rag_chain.invoke("what is cancer").content

'Cancer is a group of diseases characterized by the uncontrolled growth and spread of abnormal cells in the body. [Source: National Cancer Institute (NCI)]\n\nThere are over 100 different types of cancer, which can arise from various parts of the body, including the skin, lungs, breast, colon, prostate, and more. Cancer can be divided into several categories, including:\n\n1. Carcinomas: Originating from epithelial cells, which form the lining of organs and glands. Examples include breast cancer, lung cancer, and colon cancer. [Source: American Cancer Society (ACS)]\n2. Sarcomas: Arising from connective tissue, such as bone, cartilage, and fat. Examples include osteosarcoma (bone cancer) and soft tissue sarcoma. [Source: ACS]\n3. Leukemias: Affecting the blood and bone marrow, including acute and chronic leukemia. [Source: NCI]\n4. Lymphomas: Affecting the immune system, including Hodgkin lymphoma and non-Hodgkin lymphoma. [Source: NCI]\n\nThe exact causes of cancer are not yet fully u